In [ ]:
from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
compartments = ["cleared", "stable", "L", "I"]

model = CompartmentalModel(
    times=(0, 2000),
    compartments=compartments,
    infectious_compartments=(),
)
model.set_initial_population(
    distribution={
        "L": 100
    },
)

model.add_transition_flow(
    name="stabilisation",
    fractional_rate=Parameter("stabilisation_rate"),
    source="L",
    dest="stable",
)

model.add_transition_flow(
    name="destabilisation",
    fractional_rate=Parameter("destabilisation_rate"),
    source="stable",
    dest="L",
)

model.add_transition_flow(
    name="clearance",
    fractional_rate=Parameter("clearance_rate"),
    source="stable",
    dest="cleared",
)

model.add_transition_flow(
    name="activation",
    fractional_rate=Parameter("activation_rate"),
    source="L",
    dest="I",
)

model.request_output_for_compartments(name="active", compartments=["I"])
# # transition flows
# for i, (first_comp, second_comp) in enumerate(zip(compartments[:-2], compartments[1:-1])):
#     # add forward progression flow
#     model.add_transition_flow(
#         name=f"progress_{i + 1}",
#         fractional_rate=Parameter("progression_rate") * n_infected_comps,
#         source=first_comp,
#         dest=second_comp,
#     )
#     # add backward transition flow
#     model.add_transition_flow(
#         name=f"regress_{i + 1}",
#         fractional_rate=Parameter("regression_rate") * n_infected_comps,
#         source=second_comp,
#         dest=first_comp
#     )


# # Split infected compartments into two categories
# midpoint = (n_infected_comps + 1) // 2  # ensures the first half is larger if n_comps is odd
# model.request_output_for_compartments("early_infections", compartments[:midpoint])
# model.request_output_for_compartments("late_infections", compartments[midpoint:-1])



In [ ]:
model.get_input_parameters()

In [ ]:
parameters = {
    "activation_rate": 0.0015,
    "stabilisation_rate": .035,
    "destabilisation_rate": 0.004,
    "clearance_rate": 0.004,
}

model.run(parameters)
model.get_outputs_df()['I'].plot()

In [ ]:
# From Plot Digitiser
times = [37.681116441846726, 110.14490907569247, 684.0581706190745, 1347.8260653230773, 1985.507201667693]
active = [2.77852236374588, 6.362416253520102, 9.684563775206678, 10.02684570601338, 10.187919250839851]

from estival import priors as esp
from estival import targets as est
from estival.model import BayesianCompartmentalModel
import pandas as pd

target = est.NormalTarget(
            "active", 
            data= pd.Series(data=active, index=[round(x) for x in times]), 
            stdev=.5,  # 4.sd = 95%CI
        )

priors = [
    esp.UniformPrior("activation_rate", [.00001, .005]),
    esp.UniformPrior("stabilisation_rate", [.00001, .05]),
    esp.UniformPrior("destabilisation_rate", [.00001, .05]),
    esp.UniformPrior("clearance_rate", [.00001, .005])
]

bcm = BayesianCompartmentalModel(model,parameters, priors, [target])

In [ ]:
from tbh.runner_tools import run_metropolis_calibration, run_full_runs

In [ ]:
idata = run_metropolis_calibration(bcm, draws=5000, tune=1000, cores=4,
    chains=4,
    method="DEMetropolisZ",)

In [ ]:
from tbh import plotting as pl

burn_in = 2500
output_folder = None
pl.plot_traces(idata, burn_in, output_folder)
pl.plot_post_prior_comparison(idata, burn_in, list(bcm.priors.keys()), list(bcm.priors.values()), n_col=4, output_folder_path=output_folder)



In [ ]:
from matplotlib import pyplot as plt

full_runs, unc_df = run_full_runs(bcm, idata, burn_in, 500)
_, ax = plt.subplots()
pl.plot_model_fit_with_uncertainty(ax, unc_df, 'active', bcm, x_min=0)
ax.set_xlabel("days")
ax.set_ylabel("% active")